In [1]:
#Importing required python packages

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import text
from sklearn.metrics import classification_report

In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3 MB 6.3 MB/s 


In [ ]:
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer

Training Data:

In [2]:
#Loading 60k training dataset

In [ ]:
data = pd.read_csv('./data.csv') 

In [ ]:
data.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT
2,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
3,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
4,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ


In [4]:
#Using beatiful soup to parse the text from the html format to text format

In [ ]:
data['Text']=data.Body.apply(lambda x: BeautifulSoup(x, 'html.parser').text)
data.head(5)

,Id,Title,Body,Tags,CreationDate,Y,Text
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE,I'm already familiar with repeating tasks ever...
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT,I am having 4 different tables like \r\nselect...
2,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ,I'd like to understand why Java 8 Optionals we...
3,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ,I am attempting to overlay a title over an ima...
4,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ,"The question is very simple, but I just could ..."


In [5]:
#Converting text to lower case

In [ ]:
data['Text']=data['Text'].str.lower()
data.head(5)

,Id,Title,Body,Tags,CreationDate,Y,Text
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE,i'm already familiar with repeating tasks ever...
1,34552974,How to get all the child records from differen...,I am having 4 different tables like \r\nselect...,<sql><sql-server>,2016-01-01 01:44:52,LQ_EDIT,i am having 4 different tables like \r\nselect...
2,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ,i'd like to understand why java 8 optionals we...
3,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ,i am attempting to overlay a title over an ima...
4,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ,"the question is very simple, but i just could ..."


In [ ]:
MAX_FEATURES = 20000
MAX_LEN = 200

In [6]:
# Initializing the tokenizer used on the text data for word embeddings layer of the neural network

In [ ]:
X=data['Text'].values

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index

LSTM Model for Data Labelling:

In [ ]:
from sklearn.model_selection import train_test_split
train,validation=train_test_split(data,test_size=0.25, random_state=55)

In [ ]:
EPOCHS = 25
BATCH_SIZE = 24
#MAX_LEN = 192

In [7]:
# Initializing the label encoder 

In [ ]:
encoder = LabelEncoder()
encoder.fit(data.Y.values)
encoded_Y_train = encoder.transform(train.Y.values)
encoded_Y_valid = encoder.transform(validation.Y.values)


x_train = train.Text.values
x_valid = validation.Text.values


y_train = np_utils.to_categorical(encoded_Y_train)
y_valid = np_utils.to_categorical(encoded_Y_valid)

In [ ]:
tokens=text.Tokenizer(num_words=MAX_FEATURES, lower=True)
tokens.fit_on_texts(list(x_train))

In [ ]:
x_train=tokens.texts_to_sequences(x_train)
x_valid=tokens.texts_to_sequences(x_valid)


x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_valid = tf.keras.preprocessing.sequence.pad_sequences(x_valid, maxlen=MAX_LEN)

In [ ]:
inputs = tf.keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(MAX_FEATURES, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
#x = layers.Flatten()(x)
#x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 2,758,019
Trainable params: 2,758,019
Non-train

In [ ]:
SGD=tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=SGD,metrics=[tf.keras.metrics.AUC()])

In [ ]:
es_cb = EarlyStopping(monitor='val_loss', min_delta=0,  patience=10, verbose=0, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [8]:
#Training the model for 25 epochs

In [ ]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_valid, y_valid),callbacks = [es_cb,reduce_lr], verbose=1)

Epoch 1/25
1903/1903 [==============================] - 397s 203ms/step - loss: 0.9261 - auc: 0.7400 - val_loss: 0.6438 - val_auc: 0.8728 - lr: 0.0100
Epoch 2/25
1903/1903 [==============================] - 385s 202ms/step - loss: 0.6052 - auc: 0.8852 - val_loss: 0.5571 - val_auc: 0.9108 - lr: 0.0100
Epoch 3/25
1903/1903 [==============================] - 386s 203ms/step - loss: 0.5633 - auc: 0.9074 - val_loss: 0.6354 - val_auc: 0.8922 - lr: 0.0100
Epoch 4/25
1903/1903 [==============================] - 386s 203ms/step - loss: 0.5210 - auc: 0.9235 - val_loss: 0.7745 - val_auc: 0.8826 - lr: 0.0100
Epoch 5/25
1903/1903 [==============================] - 385s 202ms/step - loss: 0.4914 - auc: 0.9328 - val_loss: 0.5115 - val_auc: 0.9292 - lr: 0.0100
Epoch 6/25
1903/1903 [==============================] - 385s 202ms/step - loss: 0.4726 - auc: 0.9383 - val_loss: 0.4736 - val_auc: 0.9394 - lr: 0.0100
Epoch 7/25
1903/1903 [==============================] - 385s 202ms/step - loss: 0.4725 - auc: 

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 2,758,019
Trainable params: 2,758,019
Non-tra

Saving the model

In [ ]:
from google.colab import drive

In [ ]:
my_data_dir="BDA"

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model.save("/content/gdrive/My Drive/BDA/model.h5")

In [ ]:
model.save_weights("/content/gdrive/My Drive/BDA/model_weights.h5")

In [ ]:
model.load_weights("/content/gdrive/My Drive/BDA/model_weights.h5")

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model("/content/gdrive/My Drive/BDA/model.h5")

In [ ]:
!pip install tokenizer

     |████████████████████████████████| 77 kB 4.1 MB/s 


In [ ]:
df = pd.read_csv('./BDAtest.csv') 

In [ ]:
df.head()

,Unnamed: 0,Id,Body,Title,Tags
0,0,4410447,<p>I'm considering several options for sandbox...,How to disable socket creation for a Linux pro...,<linux><sockets><sandbox>
1,1,3174358,<p>the code I'm dealing with has loops like th...,MATLAB parfor is slower than for -- what is wr...,<performance><matlab><parallel-processing><par...
2,2,15459837,<p>I want to copy a file to a server using <co...,How to get the basename of the current folder ...,<makefile><directory>
3,3,5144288,<p>We have a huge project with many submodules...,how to time (profile) maven goals in a multi-m...,<performance><maven-2><build-process><profilin...
4,4,4174832,<p>I am trying to design a view using asp.net ...,How to open .cshtml in designer view?,<views><asp.net-mvc-3>


In [ ]:
df["Body"] = df.Body.astype(str)

In [ ]:
df['Text']=df.Body.apply(lambda x: BeautifulSoup(x, 'html.parser').text)
df.head(5)

,Unnamed: 0,Id,Body,Title,Tags,Text
0,0,4410447,<p>I'm considering several options for sandbox...,How to disable socket creation for a Linux pro...,<linux><sockets><sandbox>,I'm considering several options for sandboxing...
1,1,3174358,<p>the code I'm dealing with has loops like th...,MATLAB parfor is slower than for -- what is wr...,<performance><matlab><parallel-processing><par...,the code I'm dealing with has loops like the f...
2,2,15459837,<p>I want to copy a file to a server using <co...,How to get the basename of the current folder ...,<makefile><directory>,I want to copy a file to a server using scp. B...
3,3,5144288,<p>We have a huge project with many submodules...,how to time (profile) maven goals in a multi-m...,<performance><maven-2><build-process><profilin...,We have a huge project with many submodules. A...
4,4,4174832,<p>I am trying to design a view using asp.net ...,How to open .cshtml in designer view?,<views><asp.net-mvc-3>,I am trying to design a view using asp.net mvc...


In [ ]:
labels=list(encoder.classes_)
print(np.argmax(pred), labels[np.argmax(pred)])

1 LQ_CLOSE


In [ ]:
for i in range(0,10):
  test_question = []
  test_question.append(df["Body"][i])
  seq = tokenizer.texts_to_sequences(test_question)
  padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=MAX_LEN)
  pred = model.predict(padded)
  labels=list(encoder.classes_)
  print(np.argmax(pred), labels[np.argmax(pred)])


0 HQ
1 LQ_CLOSE
1 LQ_CLOSE
1 LQ_CLOSE
1 LQ_CLOSE
0 HQ
1 LQ_CLOSE
1 LQ_CLOSE
1 LQ_CLOSE
1 LQ_CLOSE


Labeling the Data


In [ ]:
df1 = pd.read_csv("./ld0.csv")

In [ ]:
df1 = df1.dropna(axis=0, subset=['Body'])

In [ ]:
df1.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,Id_1,Reputation,Views,UpVotes,DownVotes
0,20309269,1,20309302.0,NaN,2013-12-01 06:08:27,NaN,6,28024.0,<p>I have the following piece of code which <s...,3053813,NaN,2316200.0,NaN,2013-12-01 07:06:35,2020-11-05 17:27:21,check if string exists in a file,<c#><string><streamreader>,7.0,0,4.0,NaN,NaN,3053813,63,7,0,0
1,8000009,1,NaN,NaN,2011-11-03 18:25:24,NaN,21,24535.0,<p>I'm trying to write handler for uncaught ex...,605808,NaN,4370109.0,NaN,2019-05-05 22:34:53,2019-05-05 22:34:53,Is there a way in JavaScript to listen console...,<javascript><browser><cross-browser><dom-events>,6.0,1,9.0,NaN,NaN,605808,5228,404,297,8
2,2560817,1,2561077.0,NaN,2010-04-01 13:52:12,NaN,12,4611.0,<p>I'm implementing a 2D game with ships in sp...,312586,NaN,1491.0,NaN,2010-09-28 19:09:30,2010-09-28 19:09:30,2d trajectory planning of a spaceship with phy...,<lua><2d><physics><box2d>,6.0,13,13.0,NaN,NaN,312586,47928,2490,1270,287
3,9103059,1,9103096.0,NaN,2012-02-01 20:43:03,NaN,1,122.0,<p>I'm going through a book on Java and I've b...,261355,NaN,NaN,NaN,NaN,2012-02-01 20:54:48,Learning Java—Confused About Some Code,<java>,7.0,0,NaN,NaN,NaN,261355,2061,133,98,0
4,2382927,1,2383252.0,NaN,2010-03-04 21:43:29,NaN,30,31356.0,<p>I'm running into some problems using my .NE...,5678,NaN,5678.0,NaN,2010-03-08 15:55:57,2015-06-17 03:14:17,Can I use a .NET 4.0 library in a .NET 2.0 app...,<dll><.net-2.0><.net-4.0>,7.0,2,12.0,NaN,NaN,5678,4432,1023,139,7


In [ ]:
df5["Body"].isna().values.any()

False

In [ ]:
df1 = pd.read_csv("./ld1.csv")

In [ ]:
df2 = pd.read_csv("./ld2.csv")

In [ ]:
df3 = pd.read_csv("./ld3.csv")

In [ ]:
df4 = pd.read_csv("./ld4.csv")

In [ ]:
df5 = pd.read_csv("./ld5.csv")

In [ ]:
frames = [df2, df3, df4, df5]

In [ ]:
result = pd.concat(frames)

In [ ]:
len(df5)

100049

In [ ]:
df5["Body"].isna().sum()

0

In [ ]:
df4.drop(['PostTypeId','ParentId','CreationDate', 'DeletionDate', 'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'ClosedDate', 'CommunityOwnedDate', 'Id_1'], axis=1, inplace = True)

In [ ]:
df5["Quality"] = ""

In [ ]:
df5 = df5.dropna(axis=0, subset=['Body'])

In [ ]:
df5['Body'] = df5.Body.astype(str)

In [ ]:
df5['Text']= df5.Body.apply(lambda x: BeautifulSoup(x, 'html.parser').text)
df5.head(5)

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,Id_1,Reputation,Views,UpVotes,DownVotes,Quality,Text
0,8019330,1,8019355.0,NaN,2011-11-05 09:35:01,NaN,-1,246.0,<p>My lines:</p>\n\n<pre><code>ABC: XYZ\nXYZ A...,935420,NaN,100754.0,NaN,2011-11-05 13:41:37,2011-11-07 12:49:32,"In Perl, how can I join an input line with the...",<perl>,6.0,4,NaN,NaN,NaN,935420,103,31,0,0,,My lines:\nABC: XYZ\nXYZ ABC: XYZ\nXYZ:\nABC\n...
1,47018159,1,47038356.0,NaN,2017-10-30 14:48:26,NaN,6,11290.0,<p>I've tried many of the solutions given on S...,7937441,NaN,7937441.0,NaN,2017-10-31 14:55:29,2020-05-19 14:29:46,Spring Boot/H2 Console not showing my table,<spring><spring-boot><h2>,7.0,0,2.0,NaN,NaN,7937441,1431,114,79,1,,I've tried many of the solutions given on Stac...
2,33795535,1,33795595.0,NaN,2015-11-19 04:55:45,NaN,1,174.0,<p>The idea is to create a class that can chec...,5579563,NaN,2393267.0,NaN,2015-11-19 05:20:49,2015-11-22 12:08:33,Implementing a Class Called Average as a Subcl...,<python><list><class><average>,6.0,6,NaN,NaN,NaN,5579563,31,39,1,0,,The idea is to create a class that can check i...
3,43497560,1,43497727.0,NaN,2017-04-19 13:44:38,NaN,2,6629.0,<p>I'm looking for handle my <code>Bootstrap n...,7473847,NaN,-1.0,NaN,2017-05-23 12:03:02,2017-04-19 21:01:44,Center Bootstrap navbar title,<html><css><twitter-bootstrap>,7.0,2,NaN,NaN,NaN,7473847,4996,914,617,219,,I'm looking for handle my Bootstrap navbar tit...
4,41213215,1,41223040.0,NaN,2016-12-18 21:45:18,NaN,13,25987.0,<p>So I've setup webpack and <code>webpack-dev...,2525699,NaN,2525699.0,NaN,2016-12-18 21:52:00,2020-11-04 21:00:09,Webpack dev server not auto-reloading,<javascript><webpack><webpack-dev-server>,7.0,0,2.0,NaN,NaN,2525699,3864,205,330,266,,"So I've setup webpack and webpack-dev-server, ..."


In [ ]:
df5['Text']=df5['Text'].str.lower()
df5.head(5)

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,Id_1,Reputation,Views,UpVotes,DownVotes,Quality,Text
0,8019330,1,8019355.0,NaN,2011-11-05 09:35:01,NaN,-1,246.0,<p>My lines:</p>\n\n<pre><code>ABC: XYZ\nXYZ A...,935420,NaN,100754.0,NaN,2011-11-05 13:41:37,2011-11-07 12:49:32,"In Perl, how can I join an input line with the...",<perl>,6.0,4,NaN,NaN,NaN,935420,103,31,0,0,,my lines:\nabc: xyz\nxyz abc: xyz\nxyz:\nabc\n...
1,47018159,1,47038356.0,NaN,2017-10-30 14:48:26,NaN,6,11290.0,<p>I've tried many of the solutions given on S...,7937441,NaN,7937441.0,NaN,2017-10-31 14:55:29,2020-05-19 14:29:46,Spring Boot/H2 Console not showing my table,<spring><spring-boot><h2>,7.0,0,2.0,NaN,NaN,7937441,1431,114,79,1,,i've tried many of the solutions given on stac...
2,33795535,1,33795595.0,NaN,2015-11-19 04:55:45,NaN,1,174.0,<p>The idea is to create a class that can chec...,5579563,NaN,2393267.0,NaN,2015-11-19 05:20:49,2015-11-22 12:08:33,Implementing a Class Called Average as a Subcl...,<python><list><class><average>,6.0,6,NaN,NaN,NaN,5579563,31,39,1,0,,the idea is to create a class that can check i...
3,43497560,1,43497727.0,NaN,2017-04-19 13:44:38,NaN,2,6629.0,<p>I'm looking for handle my <code>Bootstrap n...,7473847,NaN,-1.0,NaN,2017-05-23 12:03:02,2017-04-19 21:01:44,Center Bootstrap navbar title,<html><css><twitter-bootstrap>,7.0,2,NaN,NaN,NaN,7473847,4996,914,617,219,,i'm looking for handle my bootstrap navbar tit...
4,41213215,1,41223040.0,NaN,2016-12-18 21:45:18,NaN,13,25987.0,<p>So I've setup webpack and <code>webpack-dev...,2525699,NaN,2525699.0,NaN,2016-12-18 21:52:00,2020-11-04 21:00:09,Webpack dev server not auto-reloading,<javascript><webpack><webpack-dev-server>,7.0,0,2.0,NaN,NaN,2525699,3864,205,330,266,,"so i've setup webpack and webpack-dev-server, ..."
